In [ ]:
# セル① : Drive を /content/drive にマウント
from google.colab import drive
drive.mount("/content/drive")        # 認証ダイアログが出るので許可


Mounted at /content/drive


In [ ]:
%%bash
set -e                                    # 途中失敗で停止
export DEBIAN_FRONTEND=noninteractive

# ---------- A. 必要パッケージ ----------
apt-get update -qq
apt-get install -y -qq build-essential cmake git libopenblas-dev liblapack-dev

# ---------- B. dlib (CUDA 無し) ----------
if [ ! -f /usr/local/lib/libdlib.a ]; then
  echo "▶ building dlib..."
  rm -rf /tmp/dlib
  git clone --depth 1 https://github.com/davisking/dlib.git /tmp/dlib
  cd /tmp/dlib
  mkdir build && cd build
  cmake -DDLIB_USE_CUDA=OFF -DUSE_AVX_INSTRUCTIONS=ON \
        -DCMAKE_INSTALL_PREFIX=/usr/local ..
  cmake --build . -j$(nproc)
  make install
  ldconfig
else
  echo "✓ dlib already installed → skip"
fi

# ---------- C. OpenFace 取得 ----------
if [ ! -d /content/OpenFace ]; then
  echo "▶ cloning OpenFace..."
  git clone --depth 1 https://github.com/TadasBaltrusaitis/OpenFace.git /content/OpenFace
  cd /content/OpenFace && bash ./download_models.sh
else
  echo "✓ /content/OpenFace exists → reuse"
  cd /content/OpenFace
  # モデルが無ければ取得
  [ -d model ] || bash ./download_models.sh
fi

# ---------- D. OpenFace ビルド ----------
cd /content/OpenFace
rm -rf build
mkdir build && cd build
cmake -DDLIB_USE_CUDA=OFF ..
make -j$(nproc)

# ---------- E. 実行ファイル確認 ----------
ls -l bin/FeatureExtraction
echo "✅ OpenFace build finished"


Process is interrupted.


In [ ]:
%%bash
# ★★★ パスを必ず修正 ★★★
BASE="/content/drive/MyDrive/AffectNet/Train"            # ← 画像が入った親フォルダ
OUT="/content/drive/MyDrive/AffectNet/OpenFace_AU"       # ← CSV の保存先

FE_BIN=$(find /content/OpenFace -type f -name FeatureExtraction | head -n1)
mkdir -p "$OUT"

classes=(anger contempt disgust fear happy neutral sad surprise)

for cls in "${classes[@]}"; do
  echo "▶ $cls"
  "$FE_BIN" -fdir "$BASE/$cls" \
            -aus -csv \
            -out_dir "$OUT/$cls" \
            -q
done

echo "✅ AU extraction done"


▶ anger
▶ contempt
▶ disgust
▶ fear
▶ happy
▶ neutral
▶ sad
▶ surprise
✅ AU extraction done


bash: line 12: : command not found
bash: line 12: : command not found
bash: line 12: : command not found
bash: line 12: : command not found
bash: line 12: : command not found
bash: line 12: : command not found
bash: line 12: : command not found
bash: line 12: : command not found


In [ ]:

# セル④ : CSV → Parquet 変換
import pandas as pd, pathlib, glob

CSV_ROOT = pathlib.Path("/content/drive/MyDrive/AffectNet/OpenFace_AU")   # ★要修正
PARQUET  = "/content/drive/MyDrive/AffectNet/affectnet_au.parquet"        # 出力

rows = []
for csv in CSV_ROOT.rglob("*.csv"):
    df = pd.read_csv(csv)
    au_cols = [c for c in df.columns if c.startswith("AU")]
    if not au_cols:          # 顔検出失敗画像をスキップ
        continue
    row = df.iloc[0][au_cols]
    row["file"] = str(csv.with_suffix(".jpg").relative_to(CSV_ROOT.parent))
    rows.append(row)

pd.DataFrame(rows).to_parquet(PARQUET, index=False)
print("✓ Parquet saved:", PARQUET, "rows:", len(rows))


✓ Parquet saved: /content/drive/MyDrive/AffectNet/affectnet_au.parquet rows: 8


In [ ]:
import pandas as pd
df = pd.read_parquet("/content/drive/MyDrive/AffectNet/affectnet_au.parquet")  # ★要修正
print(df.head())
print("総行数:", len(df))


   AU01_r  AU02_r  AU04_r  AU05_r  AU06_r  AU07_r  AU09_r  AU10_r  AU12_r  \
0    0.00    0.00    0.00     0.0    0.53    1.83    0.00    0.00    0.31   
1    0.01    0.84    0.00     0.0    0.86    0.00    0.38    0.34    0.78   
2    0.00    0.00    0.00     0.0    0.32    0.54    0.00    1.29    0.35   
3    0.00    0.94    0.00     0.0    0.04    0.00    0.00    0.91    0.62   
4    0.02    0.00    0.38     0.0    2.08    2.62    0.66    0.78    1.12   

   AU14_r  ...  AU14_c  AU15_c  AU17_c  AU20_c  AU23_c  AU25_c  AU26_c  \
0    0.73  ...     1.0     0.0     0.0     0.0     0.0     0.0     0.0   
1    1.89  ...     1.0     1.0     0.0     1.0     0.0     0.0     0.0   
2    0.57  ...     1.0     0.0     1.0     1.0     1.0     0.0     0.0   
3    0.40  ...     0.0     0.0     0.0     0.0     0.0     1.0     0.0   
4    0.00  ...     1.0     0.0     1.0     0.0     0.0     0.0     0.0   

   AU28_c  AU45_c                               file  
0     0.0     0.0        OpenFace_AU/

In [ ]:
%%bash
# ★★★ パスを必ず修正 ★★★
TEST="/content/drive/MyDrive/AffectNet/Test"            # ← 画像が入った親フォルダ
OUT="/content/drive/MyDrive/AffectNet/OpenFace_AU_Test"       # ← CSV の保存先

FE_BIN=$(find /content/OpenFace -type f -name FeatureExtraction | head -n1)
mkdir -p "$OUT"

classes=(anger contempt disgust fear happy neutral sad surprise)

for cls in "${classes[@]}"; do
  echo "▶ $cls"
  "$FE_BIN" -fdir "$TEST/$cls" \
            -aus -csv \
            -out_dir "$OUT/$cls" \
            -q
done

echo "✅ AU extraction done"


▶ Anger
▶ Contempt
▶ disgust
▶ fear
▶ happy
▶ neutral
▶ sad
▶ surprise
✅ AU extraction done


bash: line 12: : command not found
bash: line 12: : command not found
bash: line 12: : command not found
bash: line 12: : command not found
bash: line 12: : command not found
bash: line 12: : command not found
bash: line 12: : command not found
bash: line 12: : command not found


In [ ]:
# セル④ : CSV → Parquet 変換
import pandas as pd, pathlib, glob

CSV_ROOT = pathlib.Path("/content/drive/MyDrive/AffectNet/OpenFace_AU_Test")   # ★要修正
PARQUET  = "/content/drive/MyDrive/AffectNet/affectnet_test_au.parquet"        # 出力

rows = []
for csv in CSV_ROOT.rglob("*.csv"):
    df = pd.read_csv(csv)
    au_cols = [c for c in df.columns if c.startswith("AU")]
    if not au_cols:          # 顔検出失敗画像をスキップ
        continue
    row = df.iloc[0][au_cols]
    row["file"] = str(csv.with_suffix(".jpg").relative_to(CSV_ROOT.parent))
    rows.append(row)

pd.DataFrame(rows).to_parquet(PARQUET, index=False)
print("✓ Parquet saved:", PARQUET, "rows:", len(rows))


✓ Parquet saved: /content/drive/MyDrive/AffectNet/affectnet_test_au.parquet rows: 8


In [ ]:
import pandas as pd
df_test = pd.read_parquet("/content/drive/MyDrive/AffectNet/affectnet_test_au.parquet")  # ★要修正
print(df_test.head())
print("総行数:", len(df_test))

   AU01_r  AU02_r  AU04_r  AU05_r  AU06_r  AU07_r  AU09_r  AU10_r  AU12_r  \
0    1.03    0.00     0.0     0.0    0.00    0.70    0.00    0.00    0.20   
1    0.00    0.07     0.5     0.0    1.17    1.18    1.29    0.45    1.02   
2    0.00    0.00     0.0     0.0    0.37    0.51    0.00    0.76    0.33   
3    0.00    0.00     0.0     0.0    0.21    1.07    0.00    0.00    0.10   
4    0.00    0.00     0.0     0.0    1.71    0.62    0.00    2.19    2.12   

   AU14_r  ...  AU14_c  AU15_c  AU17_c  AU20_c  AU23_c  AU25_c  AU26_c  \
0    0.00  ...     0.0     0.0     0.0     0.0     0.0     1.0     1.0   
1    0.48  ...     1.0     1.0     1.0     0.0     1.0     1.0     0.0   
2    0.19  ...     1.0     0.0     1.0     0.0     0.0     0.0     0.0   
3    0.00  ...     0.0     0.0     0.0     1.0     1.0     1.0     0.0   
4    0.35  ...     1.0     0.0     0.0     1.0     0.0     1.0     1.0   

   AU28_c  AU45_c                                    file  
0     0.0     0.0        OpenFac